# Import related pacakages

In [23]:
import cobra
import sys
sys.path.append(r'./code/')
from cobrapy_ec_model_function import *

# Inputing files

In [24]:
# The genome-scale metabolic model for constructing the enzyme-constrained model
model_name = './model/iML1515_new.xml' 

# Reaction-kcat file.#s-1
# eg. AADDGT,49389.2889
#reaction_kcat_file = "./data/reaction_autopacmen_ori_noadj.csv"
#reaction_kcat_file = "./data/reaction_kappori.csv"
#reaction_kcat_file = "./data/reaction_autopacmen_ori_adj.csv"
reaction_kcat_file = "./data/reaction_autopacmen_ori_adj_change.csv"

# Gene-abundance file. 
# eg. b0789,1.1
gene_abundance_file = "./data/gene_abundance.csv"

# Gene-molecular_weight file. 
# eg. b0001,thrL,2.13846
gene_molecular_weight_file = "./data/gene_molecular_weight.csv"


# Step1: Preprocessing of model

The reversible reactions in the GEM model are divided into two irreversible reactions and splited isoenzyme. The input is iML1515 with 2712 reactions. The output is a model with 3375 irreversible reactions.

In [25]:
model = cobra.io.read_sbml_model(model_name)
convert_to_irreversible(model)
#split isoenzyme
model = isoenzyme_split_only(model)
model

Name,iML1515
Memory address,0x01a48bc86ba8
Number of metabolites,1877
Number of reactions,5883
Number of groups,38
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


# Step2: Retrieving enzyme kinetics and proteomics data

The inputs are GEM model. The outputs are 'genes' and 'gpr_relationship' data in the iML1515.

In [26]:
[genes,gpr_relationship] = get_genes_and_gpr(model)

Get the molecular weight of the enzyme (MW) according to the file of all_reaction_GPR.csv, which obtained from the previous step (gpr_relationship, ./analysis/all_reaction_GPR.csv). We need to manually correct the error of the gene_reaction_rule of a small amount of reactions in iML1515 (See Supplementary Table S1 for details), and also need to manually get the subunit of each protein from EcoCyc.

In [27]:
reaction_gene_subunit_MW = pd.read_csv("./data/reaction_gene_subunit_MW_20201123.csv", index_col=0)
reaction_gene_subunit_MW_split_only(reaction_gene_subunit_MW)

In [28]:
# reaction-gene-subunit-MW file. 
# eg. ALATA_D2,D-alanine transaminase,b2551 or b0870,45.31659 or 36.49471 ,2 or 4 
reaction_gene_subunit_MW = "./data/reaction_gene_subunit_MW.csv"
reaction_mw = calculate_reaction_mw(reaction_gene_subunit_MW)

Calculate kcat/MW. The inputs are 'reaction_kcat' and 'reaction_MW' data for calculating the kcat/MW (When the reaction is catalyzed by several isozymes, the maximum is retained).

In [29]:
save_file="./analysis/reaction_kcat_mw.csv"
reaction_kcat_mw = calculate_reaction_kcat_mw(reaction_kcat_file, reaction_mw, save_file)

Calculate f. The input is 'genes' data, 'gene_abundance.csv' and 'gene_molecular_weight.csv'.

In [30]:
f = calculate_f(genes, gene_abundance_file, gene_molecular_weight_file)
f

0.4059986079578236

# Step3: Save enzyme concentration constraint model as json file.

In [31]:
model_file = './model/iML1515_new.xml'
#PDH,1459824,608.26,2400
#SUCOAS_reverse,1447200,142.3403,10167.1839
reaction_kcat_mw_file="./analysis/reaction_kcat_mw.csv"
json_output_file="./model/iML1515_irr_enz_constraint_kapp.json"
#The enzyme mass fraction 
f = 0.406
# The total protein fraction in cell.
ptot = 0.56 
# The approximated average saturation of enzyme.
sigma = 1 #kapp data sigma is 1
# Lowerbound  of enzyme concentration constraint. 
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model_split_isoenzyme_only(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

In [32]:
json_model_path = "./model/iML1515_irr_enz_constraint_kapp.json"
fluxes_outfile = './analysis/ECMpy_kappori_solution_df_pfba.csv'
reaction_kcat_mw_file="./analysis/reaction_kcat_mw.csv"
enz_model=get_enzyme_constraint_model(json_model_path)
enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,fluxes_outfile,reaction_kcat_mw_file)
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

norm_model = cobra.io.json.load_json_model(json_model_path)
norm_model_pfba_solution = cobra.flux_analysis.pfba(norm_model)
norm_model_pfba_solution_df = norm_model_pfba_solution.to_frame()
norm_model_pfba_solution_df.to_csv('./analysis/Orimodel_solution_df_pfba.csv')
print(norm_model_pfba_solution_df.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

0.6333359344090235
0.869772642032012


# Step4: Calibration parameters

Compare with C13 data

In [33]:
import re
c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)

enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = []
for eachreaction in enz_model_pfba_solution_select.index:
    if re.search('_num',eachreaction):
        enz_model_pfba_solution_select_id.append(eachreaction.split('_num')[0])
    else:
        enz_model_pfba_solution_select_id.append(eachreaction)

c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)

['AKGDH', 'PPC', 'TPI', 'ME2', 'PFK', 'FBA', 'PGI', 'PYK', 'TKT2', 'SUCOAS_reverse']


Select calibration reaction accrording biomass diff

In [34]:
reaction_kcat_mw_file = "./analysis/reaction_kcat_mw.csv"
json_model_path = "./model/iML1515_irr_enz_constraint_kapp.json"
enzyme_amount = upperbound
percentage = 0.1#10%
reaction_biomass_outfile = './analysis/reaction_biomass.csv'
select_percentage = 0.8
df_reaction_select = select_calibration_reaction_by_biomass(reaction_kcat_mw_file, json_model_path, enzyme_amount, percentage, reaction_biomass_outfile, select_percentage)
df_reaction_select

,biomass_enz,biomass_orimodel,biomass_diff,biomass_diff_ratio
KARA1_reverse,0.619167,0.869773,0.250606,0.288128


In [35]:
if isinstance(df_reaction_select, pd.DataFrame):
    need_change_reaction=list(df_reaction_select.index)
    reaction_kcat_mw_file = "./analysis/reaction_kcat_mw.csv"
    reaction_kapp_change_file = "./analysis/reaction_kapp_change.csv"
    kcat_data_colect_file="./analysis/reaction_max_df.csv"
    change_reaction_list_round1=change_reaction_kcat_by_database_kapp_g(need_change_reaction,kcat_data_colect_file,reaction_kcat_mw_file,reaction_kapp_change_file)
    print(change_reaction_list_round1)

['KARA1_reverse']


In [38]:
model_file = './model/iML1515_new.xml'
if isinstance(df_reaction_select, pd.DataFrame):
    reaction_kcat_mw_file="./analysis/reaction_kapp_change.csv"
else:
    reaction_kcat_mw_file = "./analysis/reaction_kcat_mw.csv"
json_output_file="./model/iML1515_irr_enz_constraint_adj_round1.json"
#The enzyme mass fraction 
f = 0.406
# The total protein fraction in cell.
ptot = 0.56 
# The approximated average saturation of enzyme.
sigma = 1
#sigma = 1#kcapp数据，饱和度为1
# Lowerbound  of enzyme concentration constraint. 
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model_split_isoenzyme_only(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

In [39]:
json_model_path = "./model/iML1515_irr_enz_constraint_adj_round1.json"
fluxes_outfile= './analysis/ECMpy_kapp_adj_round1_solution_df_pfba.csv'
if isinstance(df_reaction_select, pd.DataFrame):
    reaction_kcat_mw_file="./analysis/reaction_kapp_change.csv"
else:
    reaction_kcat_mw_file = "./analysis/reaction_kcat_mw.csv"
enz_model=get_enzyme_constraint_model(json_model_path)
enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,fluxes_outfile,reaction_kcat_mw_file)
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

0.6871016739244613


In [40]:
import re
c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)

enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = []
for eachreaction in enz_model_pfba_solution_select.index:
    if re.search('_num',eachreaction):
        enz_model_pfba_solution_select_id.append(eachreaction.split('_num')[0])
    else:
        enz_model_pfba_solution_select_id.append(eachreaction)

c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)

['AKGDH', 'PPC', 'TPI', 'ME2', 'PGI', 'PYK', 'SUCOAS_reverse']


Calibration kcat according c13 reaction list

In [41]:
c13reaction_file = './data/C13reaction.csv' 
reaction_kcat_mw_file = "./analysis/reaction_kcat_mw.csv"
f = 0.406
ptot = 0.56 
sigma = 1 
enzyme_amount = round(ptot * f * sigma, 3)

c13reaction_selecet=select_calibration_reaction_by_c13(reaction_kcat_mw_file, c13reaction_file, enzyme_amount, 0.1, sigma)
print(c13reaction_selecet)

[]


In [42]:
if isinstance(df_reaction_select, pd.DataFrame):
    reaction_kcat_mw_file="./analysis/reaction_kapp_change.csv"
else:
    reaction_kcat_mw_file = "./analysis/reaction_kcat_mw.csv"

reaction_kapp_change_file = "./analysis/reaction_kapp_change_c13.csv"
#c13reaction_selecet=['CS','ACONTa','ACONTb','ICDHyr','MALS', 'MDH', 'ICL', 'SUCOAS_reverse', 'SUCDi', 'AKGDH']
kcat_data_colect_file="./analysis/reaction_max_df.csv"
change_reaction_list_round1=change_reaction_kcat_by_database_kapp_g(c13reaction_selecet,kcat_data_colect_file,reaction_kcat_mw_file,reaction_kapp_change_file)
print(change_reaction_list_round1)

[]


In [43]:
model_file = './model/iML1515_new.xml'
reaction_kcat_mw_file="./analysis/reaction_kapp_change_c13.csv"
json_output_file="./model/iML1515_irr_enz_constraint_adj_round2.json"
f = 0.406
ptot = 0.56 
sigma = 1
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model_split_isoenzyme_only(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

In [44]:
json_model_path = "./model/iML1515_irr_enz_constraint_adj_round2.json"
fluxes_outfile = './analysis/ECMpy_kapp_adj_round2_solution_df_pfba.csv'
reaction_kcat_mw_file="./analysis/reaction_kapp_change_c13.csv"
enz_model=get_enzyme_constraint_model(json_model_path)
enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,fluxes_outfile,reaction_kcat_mw_file)
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

0.6871016739244613


In [45]:
import re
c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)

enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = []
for eachreaction in enz_model_pfba_solution_select.index:
    if re.search('_num',eachreaction):
        enz_model_pfba_solution_select_id.append(eachreaction.split('_num')[0])
    else:
        enz_model_pfba_solution_select_id.append(eachreaction)

c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)

['AKGDH', 'PPC', 'TPI', 'ME2', 'PGI', 'PYK', 'SUCOAS_reverse']


Calibration kcat according Enzyme usage 

In [46]:
f = 0.406
ptot = 0.56 
sigma = 1
enz_total = round(ptot * f * sigma, 3)
reaction_flux_file = './analysis/ECMpy_kapp_adj_round2_solution_df_pfba.csv' 
reaction_kcat_mw_file = "./analysis/reaction_kapp_change_c13.csv"
reaction_enz_usage_file = './analysis/ECMpy_kapp_adj_round2_reaction_enz_usage_df.csv'

reaction_enz_usage_df = get_enzyme_usage(enz_total,reaction_flux_file,reaction_kcat_mw_file,reaction_enz_usage_file)
reaction_enz_usage_df.head(22)

,kcat_mw,flux,enz useage,enz ratio
ATPS4rpp_num2,785.767390,59.139755,0.075264,0.331558
NADH16pp,1406.082994,33.934437,0.024134,0.106317
PSERT,66.819799,1.142774,0.017102,0.075341
ACCOAC,122.895876,1.658211,0.013493,0.059440
PDH,2400.000000,13.021821,0.005426,0.023902
PAPSR2_num2,33.787531,0.170045,0.005033,0.022171
KARA2,42.209426,0.199623,0.004729,0.020834
CS,590.550663,2.772434,0.004695,0.020681
ACGS,64.093962,0.203239,0.003171,0.013969
ENO,2525.688868,7.802145,0.003089,0.013608


In [47]:
select_reaction = list(reaction_enz_usage_df[reaction_enz_usage_df['enz ratio']>0.01].index)#more than 1%
print(select_reaction)
reaction_kcat_mw_file = "./analysis/reaction_kapp_change_c13.csv"
reaction_kapp_change_file = "./analysis/reaction_kapp_change_c13_enzuse.csv"
kcat_data_colect_file="./analysis/reaction_max_df.csv"

change_reaction_list_round1=change_reaction_kcat_by_database_kapp_g(select_reaction,kcat_data_colect_file,reaction_kcat_mw_file,reaction_kapp_change_file)
print(change_reaction_list_round1)

['ATPS4rpp_num2', 'NADH16pp', 'PSERT', 'ACCOAC', 'PDH', 'PAPSR2_num2', 'KARA2', 'CS', 'ACGS', 'ENO', 'GAPD', 'SADT2', 'ADSS', 'PGL', 'IPPMIa_reverse', 'IPPMIb_reverse']
['ATPS4rpp_num2', 'NADH16pp', 'PSERT', 'ACCOAC', 'PAPSR2_num2', 'KARA2', 'CS', 'ACGS', 'ENO', 'SADT2', 'ADSS', 'IPPMIa_reverse', 'IPPMIb_reverse']


In [48]:
model_file = './model/iML1515_new.xml'
reaction_kcat_mw_file="./analysis/reaction_kapp_change_c13_enzuse.csv"
json_output_file="./model/iML1515_irr_enz_constraint_adj_round3.json"
f = 0.406
ptot = 0.56 
sigma = 1
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model_split_isoenzyme_only(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

In [49]:
json_model_path = "./model/iML1515_irr_enz_constraint_adj_round3.json"
fluxes_outfile = './analysis/ECMpy_kapp_adj_round3_solution_df_pfba.csv'
reaction_kcat_mw_file="./analysis/reaction_kapp_change_c13_enzuse.csv"
enz_model=get_enzyme_constraint_model(json_model_path)
enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,fluxes_outfile,reaction_kcat_mw_file)
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

0.8697726420320072


In [50]:
import re
c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)

enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = []
for eachreaction in enz_model_pfba_solution_select.index:
    if re.search('_num',eachreaction):
        enz_model_pfba_solution_select_id.append(eachreaction.split('_num')[0])
    else:
        enz_model_pfba_solution_select_id.append(eachreaction)

c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)

['TALA', 'PTAr', 'ME2', 'ICL', 'ACKr_reverse', 'PYK', 'PFL']


In [54]:
reaction_kcat_mw_file = "./analysis/reaction_kapp_change_c13_enzuse.csv"
reaction_kapp_change_file = "./analysis/reaction_kapp_change_c13_enzuse_manual.csv"
select_reaction=['PDH']
#c13reaction_selecet=['CS','ACONTa','ACONTb','ICDHyr','MALS', 'MDH', 'ICL', 'SUCOAS_reverse', 'SUCDi', 'AKGDH']
change_fold=[10]
change_reaction_list_round1=change_reaction_kcat_by_foldlist(select_reaction,change_fold,reaction_kcat_mw_file,reaction_kapp_change_file)
print(change_reaction_list_round1)

model_file = './model/iML1515_new.xml' 
reaction_kcat_mw_file="./analysis/reaction_kapp_change_c13_enzuse_manual.csv"
json_output_file="./model/iML1515_irr_enz_constraint_adj_manual.json"
#The enzyme mass fraction 
f = 0.406
# The total protein fraction in cell.
ptot = 0.56 
# The approximated average saturation of enzyme.
sigma = 1#0.5 
# Lowerbound  of enzyme concentration constraint. 
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model_split_isoenzyme_only(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

json_model_path = "./model/iML1515_irr_enz_constraint_adj_manual.json"
fluxes_outfile = './analysis/ECMpy_kapp_adj_manual_solution_df_pfba.csv'
reaction_kcat_mw_file="./analysis/reaction_kapp_change_c13_enzuse_manual.csv"
enz_model=get_enzyme_constraint_model(json_model_path)

enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,fluxes_outfile,reaction_kcat_mw_file)
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)

enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = []
for eachreaction in enz_model_pfba_solution_select.index:
    if re.search('_num',eachreaction):
        enz_model_pfba_solution_select_id.append(eachreaction.split('_num')[0])
    else:
        enz_model_pfba_solution_select_id.append(eachreaction)
c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)

['PDH']
0.8234989839795128
['TKT2', 'ICL', 'ACKr_reverse', 'PTAr', 'ME2']


# Step4: Solveing enzyme concentration constraint by COBRApy.

In [21]:
#run enzyme constraint metabolic model
json_model_path="./model/iML1515_irr_enz_constraint_adj_round2.json"
enz_model=get_enzyme_constraint_model(json_model_path)
pfba_solution = cobra.flux_analysis.pfba(enz_model)
pfba_solution_df = pfba_solution.to_frame()
#pfba_solution_df.to_csv('./analysis/ECMpy_solution_df_pfba.csv')
pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M']

0.7901116510198356

In [22]:
#run genome-scale metabolic model
norm_model=cobra.io.json.load_json_model(json_model_path)
pfba_solution = cobra.flux_analysis.pfba(norm_model)
pfba_solution_df = pfba_solution.to_frame()
#pfba_solution_df.to_csv('./analysis/Orimodel_solution_df_pfba.csv')
pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M']


0.8769972144269698